In [1]:
import pandas as pd
from sklearn.model_selection import cross_validate, cross_val_score,StratifiedKFold
from sklearn.preprocessing import LabelEncoder


from sklearn import tree
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier

In [2]:
# Define your models
model = tree.DecisionTreeClassifier()

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [8]:
df_train = pd.read_csv("../data/train.csv", delimiter=",")
df_test = pd.read_csv("../data/test.csv", delimiter=",")
df_score = df_test.copy()

In [9]:
df_score

,user,time
0,0,2024-04-07
1,0,2024-04-08
2,0,2024-04-09
3,0,2024-04-10
4,0,2024-04-11
...,...,...
3858,32,2024-07-26
3859,32,2024-07-27
3860,32,2024-07-28
3861,32,2024-07-29


In [18]:
df_train[(df_train['user']==8)&(df_train['bought']==1)]

,user,time,bought,day_of_week,day_of_year,month,day_of_month
3816,8,2023-04-27,1,3,117,4,27
3822,8,2023-05-03,1,2,123,5,3
3835,8,2023-05-16,1,1,136,5,16
3836,8,2023-05-17,1,2,137,5,17
3852,8,2023-06-02,1,4,153,6,2
4018,8,2023-11-15,1,2,319,11,15


In [11]:
data_set = ["df_train","df_score"]

for dfname in data_set: 
    df=globals()[dfname]
    df['time'] = pd.to_datetime(df['time'])
    df['day_of_week'] = pd.to_datetime(df['time']).dt.weekday
    # df['week_of_year'] = pd.to_datetime(df['time']).dt.weekofyear
    df['day_of_year'] = pd.to_datetime(df['time']).dt.dayofyear
    df['month'] = pd.to_datetime(df['time']).dt.month
    df['day_of_month'] = pd.to_datetime(df['time']).dt.day
    globals()[dfname]=df

In [12]:
df_test
# ['time'].unique()

,user,time
0,0,2024-04-07
1,0,2024-04-08
2,0,2024-04-09
3,0,2024-04-10
4,0,2024-04-11
...,...,...
3858,32,2024-07-26
3859,32,2024-07-27
3860,32,2024-07-28
3861,32,2024-07-29


In [13]:
user_list = df_train["user"].unique()
user_list

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])

In [14]:
df_test[df_test['user']==32]

,user,time
3748,32,2024-04-07
3749,32,2024-04-08
3750,32,2024-04-09
3751,32,2024-04-10
3752,32,2024-04-11
...,...,...
3858,32,2024-07-26
3859,32,2024-07-27
3860,32,2024-07-28
3861,32,2024-07-29


In [30]:
y_pred = pd.DataFrame
submission = pd.DataFrame(columns=['user','time','bought'])
for user_x in user_list:
    X=df_train[df_train['user']==user_x][['day_of_week','day_of_month','month','day_of_year']]
    y=df_train[df_train['user']==user_x]['bought']
    x_score=df_score[df_score['user']==user_x][['day_of_week','day_of_month','month','day_of_year']]

    print(X.shape)
    print(y.shape)
    print(x_score.shape)

    scores_stratified = cross_val_score(model, X, y, cv=skf, scoring='f1') # 'f1' is often a good choice for imbalanced data
    print(f"User {user_x} Stratified Cross-validation scores:", scores_stratified)

    model.fit(X, y)
    y_pred = model.predict(x_score)

    sub = pd.Series(y_pred, name='bought')
    # sub = pd.DataFrame(sub)
    print(sub)
    # submission = pd.concat(submission, sub, axis=0)

(461, 4)
(461,)
(115, 4)
User 0 Stratified Cross-validation scores: [1.         0.97777778 1.        ]
0      0
1      0
2      0
3      1
4      0
      ..
110    0
111    0
112    0
113    0
114    0
Name: bought, Length: 115, dtype: int64
(461, 4)
(461,)
(115, 4)
User 1 Stratified Cross-validation scores: [0.25       0.13636364 0.23076923]
0      0
1      0
2      0
3      0
4      0
      ..
110    0
111    0
112    0
113    0
114    1
Name: bought, Length: 115, dtype: int64
(461, 4)
(461,)
(115, 4)
User 2 Stratified Cross-validation scores: [0.45833333 0.57142857 0.38095238]
0      0
1      0
2      0
3      0
4      1
      ..
110    0
111    0
112    0
113    0
114    0
Name: bought, Length: 115, dtype: int64
(461, 4)
(461,)
(115, 4)
User 3 Stratified Cross-validation scores: [0.18181818 0.14285714 0.11111111]
0      0
1      0
2      0
3      0
4      0
      ..
110    0
111    0
112    0
113    0
114    0
Name: bought, Length: 115, dtype: int64
(464, 4)
(464,)
(115, 4)
User 4 

In [31]:
sub

0      0
1      0
2      0
3      0
4      0
      ..
110    0
111    0
112    0
113    0
114    0
Name: bought, Length: 115, dtype: int64